<a href="https://colab.research.google.com/github/Anasj94/AI-recipe/blob/main/Training_Model_working_great.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW

In [4]:
# Define your dataset class
class RecipeDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data.iloc[idx]
        input_text = f"NER: {sample['NER']} Recipe: {sample['title']} Ingredients: {sample['ingredients']} Directions: {sample['directions']}"
        target_text = sample['title']  # You may modify this based on your output requirement
        return input_text, target_text

In [6]:
# Load and preprocess your dataset
df = pd.read_csv('/content/Recipe.csv', engine='python', on_bad_lines='skip')
dataset = RecipeDataset(df)


In [8]:
print(df.head())

   Unnamed: 0                  title  \
0           0    No-Bake Nut Cookies   
1           1  Jewell Ball'S Chicken   
2           2            Creamy Corn   
3           3          Chicken Funny   
4           4   Reeses Cups(Candy)     

                                         ingredients  \
0  ["1 c. firmly packed brown sugar", "1/2 c. eva...   
1  ["1 small jar chipped beef, cut up", "4 boned ...   
2  ["2 (16 oz.) pkg. frozen corn", "1 (8 oz.) pkg...   
3  ["1 large whole chicken", "2 (10 1/2 oz.) cans...   
4  ["1 c. peanut butter", "3/4 c. graham cracker ...   

                                          directions  \
0  ["In a heavy 2-quart saucepan, mix brown sugar...   
1  ["Place chipped beef on bottom of baking dish....   
2  ["In a slow cooker, combine all ingredients. C...   
3  ["Boil and debone chicken.", "Put bite size pi...   
4  ["Combine first four ingredients and press in ...   

                                              link    source  \
0   www.cookbooks.com

In [ ]:
# Load pre-trained T5 model and tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Define training parameters
batch_size = 16
num_epochs = 11
learning_rate = 5e-5

# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)



In [ ]:
# Move model to GPU
model.to(device)

# Define optimizer (no need to move it to GPU explicitly)
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Define your DataLoader with increased batch size
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [19]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for batch in dataloader:
        input_texts, target_texts = batch
        input_ids = tokenizer.batch_encode_plus(input_texts, padding=True, return_tensors="pt", truncation=True)["input_ids"].to(device)
        target_ids = tokenizer.batch_encode_plus(target_texts, padding=True, return_tensors="pt", truncation=True)["input_ids"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, labels=target_ids)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(dataloader)}")

Epoch 1, Loss: 0.0012105358246493466
Epoch 2, Loss: 0.0005347824543666039
Epoch 3, Loss: 0.00037185590121744885
Epoch 4, Loss: 0.00034458497492692326
Epoch 5, Loss: 0.00021205978381655383
Epoch 6, Loss: 0.00025010333057562455
Epoch 7, Loss: 0.00019158050489586988
Epoch 8, Loss: 0.00011995644792750123
Epoch 9, Loss: 0.00014049668964283407
Epoch 10, Loss: 0.00012216119066642132
Epoch 11, Loss: 0.00014521781276909948


In [20]:
# Save the trained model
model.save_pretrained("trained_model")

In [29]:
def generate_output(ingredients, model, tokenizer, device):
    # Construct input text with placeholders for title and directions
    input_text = f"NER: {ingredients} Recipe: [Title] Ingredients: {ingredients} Directions: [Directions]"

    # Tokenize and move input text to the specified device
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    # Generate output text
    outputs = model.generate(input_ids)

    # Decode output text and return
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return output_text

# Example usage
ingredients_input = "chicken, rice, oil"
output_text = generate_output(ingredients_input, model, tokenizer, device)
print("Generated Recipe Title:")
print(output_text)


Generated Recipe Title:
[Title]
